In [ ]:
import pandas as pd
import plotly.express as px
import warnings
import numpy as np
warnings.filterwarnings('ignore')

from Materials_Data_Analytics.materials.solutes import MolecularOxygen
from Materials_Data_Analytics.materials.solvents import Solvent 
from Materials_Data_Analytics.materials.ions import Cation, Anion 
from Materials_Data_Analytics.materials.electrolytes import Electrolyte
from Materials_Data_Analytics.materials.polymers import NType
from Materials_Data_Analytics.continuum_modelling.microkinetic_modelling import ECpD 


In [ ]:
# Create the solution for our model

na_cation = Cation(name='Na+')
cl_anion = Anion(name='Cl-')
water_solvent = Solvent('water')
oxygen_solute = MolecularOxygen()

my_electrolyte = Electrolyte(solvent=water_solvent, 
                             cation=na_cation, 
                             anion=cl_anion, 
                             concentrations={na_cation: 0.1, cl_anion: 0.1, oxygen_solute: 0.0008}, 
                             solute=oxygen_solute, 
                             pH=14.2, 
                             temperature=298,
                             diffusivities={oxygen_solute: 0.000019},
                             viscosity=0.01
                             )


In [ ]:
# Create the polymer for our model
my_polymer = NType('NDI', formal_reduction_potential = -0.3159)

In [ ]:
# Create the model 
my_model = ECpD(electrolyte=my_electrolyte, polymer=my_polymer, rotation_rate=1600)

In [ ]:
# parameters
k01 = (-5.906)
kf2 = (1.0807)
kf3 = (4.688)

beta = 0.5
guess = [1, 0, 0, 0]

In [ ]:
# Calculate values across a potential sweep

data = (my_model
        .get_e_sweep(E_max=0.5, E_min=-1, E_n=200, k01=k01, beta=0.5, kf2=kf2, kf3=kf3, guess=[1, 0, 0, 0])
        .assign(potential = lambda x: x['potential'] + 0.059 * 14.2) # convert to RHE
        )

fig = px.line(data, x='potential', y=['disk_current_density', 'ring_current_density'], title='Current Densities vs Potential', 
        template='presentation', height=500, width=1000, labels={'value': 'Current density', 'potential': 'V [vs RHE]'}).show()

px.line(data, x='potential', y=['thetaN', 'thetaP'], title='Current Densities vs Potential', 
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show()

px.line(data, x='potential', y=['v1', 'v2', 'v3'], title='Current Densities vs Potential', 
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show()

px.line(data, x='potential', y=['CS02', 'CS02_superoxide'], title='Current Densities vs Potential', 
        template='presentation', height=500, width=1000, labels={'value': 'Surface Coverages', 'potential': 'V [vs RHE]'}).show()


In [ ]:
# effect of kf3

data_list = []
for k in 0.00001, 0.0001, 0.001, 0.015, 0.1, 1, 2, 3, 4, 5, 6, 7, 8:
    data = my_model.get_e_sweep(E_max=0.5, E_min=-1, E_n=200, k01=k01, beta=0.5, kf2=kf2, kf3=k, guess=[1, 0, 0, 0])
    data = data.assign(potential = lambda x: x['potential'] + 0.059 * 14.2).assign(kf3 = k) # convert to RHE
    data_list.append(data)

data = pd.concat(data_list)

px.line(data, x='potential', y='disk_current_density', color='kf3',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show()

px.line(data, x='potential', y='ring_current_density', color='kf3',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show()

px.line(data, x='potential', y='thetaN', color='kf3',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show()

px.line(data, x='potential', y='thetaP', color='kf3',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show()

px.line(data, x='potential', y='v1', color='kf3',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show()

px.line(data, x='potential', y='v2', color='kf3',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show()

px.line(data, x='potential', y='v3', color='kf3',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show()

px.line(data, x='potential', y='CS02', color='kf3',
        template='presentation', height=500, width=1000, labels={'value': 'Surface Coverages', 'potential': 'V [vs RHE]'}).show()

px.line(data, x='potential', y='CS02_superoxide', color='kf3',
        template='presentation', height=500, width=1000, labels={'value': 'Surface Coverages', 'potential': 'V [vs RHE]'}).show()
    


In [ ]:
# effect of kf2

data_list = []
for k in 0.1, 0.5, 1, 1.5, 2, 2.5:
    data = my_model.get_e_sweep(E_max=0.5, E_min=-1, E_n=200, k01=k01, beta=0.5, kf2=k, kf3=kf3, guess=[0.8, 0.2, 0, 0])
    data = data.assign(potential = lambda x: x['potential'] + 0.059 * 14.2).assign(kf2 = k) # convert to RHE
    data_list.append(data)

data = pd.concat(data_list)

px.line(data, x='potential', y='disk_current_density', color='kf2',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='ring_current_density', color='kf2',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='thetaN', color='kf2',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='thetaP', color='kf2',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='v1', color='kf2',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='v2', color='kf2',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='v3', color='kf2',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='CS02', color='kf2',
        template='presentation', height=500, width=1000, labels={'value': 'Surface Coverages', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='CS02_superoxide', color='kf2',
        template='presentation', height=500, width=1000, labels={'value': 'Surface Coverages', 'potential': 'V [vs RHE]'}).show(renderer='svg')
    

In [ ]:
# effect of k01

data_list = []
for k in -7, -5, -3, -1, 1, 3, 5:
    data = my_model.get_e_sweep(E_max=0.5, E_min=-1, E_n=200, k01=k, beta=0.5, kf2=kf2, kf3=kf3, guess=[0.8, 0.2, 0, 0])
    data = data.assign(potential = lambda x: x['potential'] + 0.059 * 14.2).assign(kf1 = k) # convert to RHE
    data_list.append(data)

data = pd.concat(data_list)

px.line(data, x='potential', y='disk_current_density', color='kf1',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='ring_current_density', color='kf1',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='thetaN', color='kf1',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='thetaP', color='kf1',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='v1', color='kf1',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='v2', color='kf1',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='v3', color='kf1',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='CS02', color='kf1',
        template='presentation', height=500, width=1000, labels={'value': 'Surface Coverages', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='CS02_superoxide', color='kf1',
        template='presentation', height=500, width=1000, labels={'value': 'Surface Coverages', 'potential': 'V [vs RHE]'}).show(renderer='svg')

In [ ]:
# Effect of rotation rate on current densities

data_list = []
for r in [20, 500, 1000, 1600, 3000, 5000]:
    model = ECpD(electrolyte=my_electrolyte, polymer=my_polymer, rotation_rate=r)
    data = model.get_e_sweep(E_max=0.5, E_min=-1, E_n=200, k01=k01, beta=0.5, kf2=kf2, kf3=kf3, guess=[1, 0, 0, 0])
    data = data.assign(potential = lambda x: x['potential'] + 0.059 * 14.2).assign(rotation_rate = r)
    data_list = data_list + [data]

data = pd.concat(data_list)

px.line(data, x='potential', y='disk_current_density', color='rotation_rate',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='ring_current_density', color='rotation_rate',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='thetaN', color='rotation_rate',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='thetaP', color='rotation_rate',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='v1', color='rotation_rate',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='v2', color='rotation_rate',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='v3', color='rotation_rate',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='CS02', color='rotation_rate',
        template='presentation', height=500, width=1000, labels={'value': 'Surface Coverages', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='CS02_superoxide', color='rotation_rate',
        template='presentation', height=500, width=1000, labels={'value': 'Surface Coverages', 'potential': 'V [vs RHE]'}).show(renderer='svg')



In [ ]:
# Effect of polymer reduction potential on current densities

data_list = []
for e in [-0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0]:
    my_polymer = NType('NDI', formal_reduction_potential = e)
    model = ECpD(electrolyte=my_electrolyte, polymer=my_polymer, rotation_rate=1600)
    data = model.get_e_sweep(E_max=0.5, E_min=-1, E_n=200, k01=k01, beta=0.5, kf2=kf2, kf3=kf3, guess=[0.99, 0.01, 0, 0])
    data = data.assign(potential = lambda x: x['potential'] + 0.059 * 14.2).assign(reduction_potential = e)
    data_list = data_list + [data]

data = pd.concat(data_list)

px.line(data, x='potential', y='disk_current_density', color='reduction_potential',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='ring_current_density', color='reduction_potential',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='thetaN', color='reduction_potential',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='thetaP', color='reduction_potential',
        template='presentation', height=500, width=1000, labels={'value': 'Population', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='v1', color='reduction_potential',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='v2', color='reduction_potential',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='v3', color='reduction_potential',
        template='presentation', height=500, width=1000, labels={'value': 'Rate constants', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='CS02', color='reduction_potential',
        template='presentation', height=500, width=1000, labels={'value': 'Surface Coverages', 'potential': 'V [vs RHE]'}).show(renderer='svg')

px.line(data, x='potential', y='CS02_superoxide', color='reduction_potential',
        template='presentation', height=500, width=1000, labels={'value': 'Surface Coverages', 'potential': 'V [vs RHE]'}).show(renderer='svg')

